In [ ]:
from dataloader import IGB260MDGLDataset, OGBDGLDataset
print("Dataset: OGB")
dataset = OGBDGLDataset(args)
g = dataset[0]
g  = g.formats('csc')

import torch
g.ndata['features'] = g.ndata['feat']
g.ndata['labels'] = g.ndata['label']

train_nid = torch.nonzero(g.ndata['train_mask'], as_tuple=True)[0]
print(torch.nonzero(g.ndata['train_mask'], as_tuple=True))
val_nid = torch.nonzero(g.ndata['val_mask'], as_tuple=True)[0]
test_nid = torch.nonzero(g.ndata['test_mask'], as_tuple=True)[0]
in_feats = g.ndata['features'].shape[1]
print(train_nid.shape, val_nid.shape, test_nid.shape)

import numpy as np
import numba as nb
from time import perf_counter
import torch

def ultra_fast_analysis(edge_path):
    """修复版高效边关系分析（兼容Numba旧版，确保正确性）"""
    print(f"\n[阶段1] 数据加载与验证")
    start_load = perf_counter()
    raw_data = np.load(edge_path)
    if raw_data.shape[0] == 2:
        edges = raw_data.T.astype(np.uint32)
    else:
        edges = raw_data[:, :2].astype(np.uint32)
    print(f"数据加载完成 | 形状：{edges.shape} | 内存占用：{edges.nbytes/1024**3:.1f}GB")
    print(f"首行样本：{edges[0]} | 末行样本：{edges[-1]}")
    print(f"加载耗时：{perf_counter() - start_load:.2f}s")

    print("\n[阶段2] 排序（正确顺序）")
    start_sort = perf_counter()
    sort_key = (edges[:, 1], edges[:, 0])
    sorted_order = np.lexsort(sort_key)
    sorted_edges = edges[sorted_order]
    print(f"排序完成 | 首行样本：{sorted_edges[0]}")
    print(f"排序耗时：{perf_counter() - start_sort:.2f}s")

    print("\n[阶段3] 邻接表构建")
    start_build = perf_counter()
    @nb.njit
    def compute_offsets(sorted_src, max_src_id):
        offsets = np.zeros(max_src_id + 2, dtype=np.uint32)
        current = sorted_src[0]
        count = 1
        for i in range(1, len(sorted_src)):
            if sorted_src[i] == current:
                count += 1
            else:
                offsets[current + 1] = count
                current = sorted_src[i]
                count = 1
        offsets[current + 1] = count
        for i in range(1, len(offsets)):
            offsets[i] += offsets[i - 1]
        return offsets
    src_nodes = sorted_edges[:, 0]
    max_src_id = int(src_nodes.max())
    offsets = compute_offsets(src_nodes, max_src_id)
    neighbors = sorted_edges[:, 1].astype(np.uint32)
    print(f"构建完成 | 总邻居数：{len(neighbors):,}")
    print(f"构建耗时：{perf_counter() - start_build:.2f}s")

    print("\n[阶段4] 统计信息生成")
    start_stat = perf_counter()
    actual_src_ids = np.unique(src_nodes)
    out_degrees = offsets[1:] - offsets[:-1]
    valid_degrees = out_degrees[actual_src_ids]
    stats = {
        "total_edges": len(edges),
        "unique_sources": int(len(actual_src_ids)),
        "max_out_degree": int(valid_degrees.max()),
        "avg_out_degree": float(valid_degrees.mean()),
        "median_out_degree": float(np.median(valid_degrees)),
        "out_degree_distribution": np.bincount(valid_degrees),
        "original_ids": actual_src_ids,
        "offsets": offsets,
        "neighbors": neighbors
    }
    print(f"统计耗时：{perf_counter() - start_stat:.2f}s")
    return stats

def add_self_edges_all(original_stats, edges, total_nodes=111059956):
    print("\n[自环处理-All] 初始化")
    start = perf_counter()
    node_count = total_nodes
    full_offsets = original_stats['offsets']
    if len(full_offsets) < node_count + 1:
        full_offsets = np.concatenate([full_offsets,
                                       np.full(node_count + 1 - len(full_offsets), full_offsets[-1], dtype=full_offsets.dtype)])
    full_neighbors = original_stats['neighbors']
    orig_counts = full_offsets[1:] - full_offsets[:-1]
    new_counts = orig_counts + 1
    new_offsets = np.empty(node_count + 1, dtype=np.uint32)
    new_offsets[0] = 0
    new_offsets[1:] = np.cumsum(new_counts)
    total_new_edges = int(new_offsets[-1])
    new_neighbors = np.empty(total_new_edges, dtype=np.uint32)
    print(f"节点总数：{node_count:,}，分配新邻居数组：{new_neighbors.size:,}")

    @nb.njit(parallel=True)
    def fill(full_offsets, full_neighbors, new_offsets, new_neighbors):
        n = len(new_offsets) - 1
        for i in nb.prange(n):
            s, e = full_offsets[i], full_offsets[i+1]
            has_self_edge = False
            for j in range(e - s):
                nbr = full_neighbors[s + j]
                if nbr == i:  # 检查是否已包含自环边
                    has_self_edge = True
                new_neighbors[new_offsets[i] + j] = nbr
            if not has_self_edge:
                new_neighbors[new_offsets[i+1] - 1] = i  # 如果没有自环，才添加自环边
        return new_neighbors
    
    adjusted_neighbors = fill(full_offsets, full_neighbors, new_offsets, new_neighbors)
    print(f"填充耗时：{perf_counter() - start:.2f}s")
    new_degrees = new_counts
    final_stats = original_stats.copy()
    final_stats.update({
        'offsets': new_offsets,
        'neighbors': adjusted_neighbors,
        'total_edges': int(original_stats['total_edges'] + node_count),
        'max_out_degree': int(new_degrees.max()),
        'avg_out_degree': float(new_degrees.mean()),
        'median_out_degree': float(np.median(new_degrees)),
        'out_degree_distribution': np.bincount(new_degrees),
        'unique_sources': int(np.count_nonzero(new_degrees)),
        'original_ids': np.arange(node_count, dtype=np.uint32)
    })
    print(f"更新统计耗时：{perf_counter() - start:.2f}s")
    return final_stats


def split_neighbors_by_remove_indices_enhanced(stats, remove_indices):
    offsets = stats['offsets']
    neighbors = stats['neighbors']
    node_count = len(offsets) - 1
    mask = np.zeros(node_count, dtype=bool)
    mask[remove_indices] = True
    in_lists, out_lists = [], []
    for i in range(node_count):
        block = neighbors[offsets[i]:offsets[i+1]]
        in_lists.append(block[mask[block]])
        out_lists.append(block[~mask[block]])
    def build(lists):
        counts = np.array([len(x) for x in lists], dtype=np.uint32)
        offs = np.zeros(node_count+1, dtype=np.uint32)
        offs[1:] = np.cumsum(counts)
        flat = np.concatenate(lists).astype(np.uint32)
        return {
            'offsets': offs,
            'neighbors': flat,
            'original_ids': np.arange(node_count, dtype=np.uint32),
            'total_edges': int(flat.size),
            'max_out_degree': int(counts.max()) if counts.size else 0,
            'avg_out_degree': float(counts.mean()) if counts.size else 0.0,
            'median_out_degree': float(np.median(counts)) if counts.size else 0.0,
            'unique_sources': int(np.count_nonzero(counts))
        }
    return build(in_lists), build(out_lists)

@nb.njit
def _grouping_stage1(
    order, offsets, neighbors, used,
    neighbor_groups, sources, group_count,
    group_size, max_groups_per_node
):
    """
    第一阶段：对每个 node 最多生成 max_groups_per_node 组，不复用邻居
    """
    for idx in range(order.shape[0]):
        node = order[idx]
        # per-node 组计数
        node_gp = 0
        s, e = offsets[node], offsets[node+1]
        tmp = np.empty(group_size, dtype=np.int32)
        cnt = 0
        for j in range(s, e):
            nbr = neighbors[j]
            if nbr < used.shape[0] and not used[nbr]:
                tmp[cnt] = nbr
                cnt += 1
                if cnt == group_size:
                    # 形成一组
                    for k in range(group_size):
                        neighbor_groups[group_count[0], k] = tmp[k]
                        used[tmp[k]] = 1
                    sources[group_count[0]] = node
                    group_count[0] += 1
                    node_gp += 1
                    cnt = 0
                    # 达到上限，跳出该 node
                    if node_gp >= max_groups_per_node:
                        break
        # 处理完当前 node，继续下一个
    # 返回时 group_count[0] 即为第一阶段实际组数


def full_optimized_grouping_multi_var_group_order(
    stats, prank_scores,
    redundancy_rate=0.2,
    group_size=4,
    max_groups_per_node=5
):
    # —— 预处理同原来，得到 offsets, neighbors, order … —— #
    # 第一阶段：不复用，每个 node 最多 5 组
    orig_off   = stats['offsets'].astype(np.int32)
    orig_nei   = stats['neighbors'].astype(np.int32)
    total_nodes = orig_off.shape[0] - 1

    # —— 预处理：去重并按 prank_scores 排序 —— #
    counts = np.empty(total_nodes, dtype=np.int32)
    for i in range(total_nodes):
        s, e = orig_off[i], orig_off[i+1]
        counts[i] = np.unique(orig_nei[s:e]).size

    offsets = np.empty_like(orig_off)
    offsets[0] = 0
    for i in range(1, total_nodes+1):
        offsets[i] = offsets[i-1] + counts[i-1]

    neighbors = np.empty(offsets[-1], dtype=np.int32)
    for i in range(total_nodes):
        s, e = orig_off[i], orig_off[i+1]
        uniq = np.unique(orig_nei[s:e])
        order_idx = np.argsort(-prank_scores[uniq], kind='stable')
        neighbors[offsets[i]:offsets[i+1]] = uniq[order_idx]


    max_groups = neighbors.size // group_size
    ng = np.full((max_groups, group_size), -1, dtype=np.int32)
    src = np.full(max_groups, -1, dtype=np.int32)
    used = np.zeros(total_nodes, dtype=np.uint8)
    gcount = np.zeros(1, dtype=np.int32)

    order = np.lexsort(
        (
            -prank_scores,  # 次排序键：pagerank 降序
            -counts         # 主排序键：neighbor count 降序
        )
    ).astype(np.int32)
    _grouping_stage1(
        order, offsets, neighbors,
        used, ng, src, gcount,
        group_size, max_groups_per_node
    )

    valid_ng  = ng[:gcount[0]]
    valid_src = src[:gcount[0]]
    missing = total_nodes - int(used.sum())
    print(f"[第一阶段] 组数: {gcount[0]}, 未覆盖: {missing}")

    # —— 第二阶段：允许重用，同样对每个 node 补到最多 5 组 —— #
    # 先统计第一阶段每个 node 已有组数
    # valid_src 是长度 gcount[0] 的一维数组
    counts_per_node = np.zeros(total_nodes, dtype=np.int32)
    for s in valid_src:
        counts_per_node[s] += 1

    new_ng = []
    new_src = []
    threshold = int(total_nodes * (1 + redundancy_rate))
    slots = valid_ng.size

    # 只遍历那些组数 < max_groups_per_node 的 node
    for node in order:
        cur = counts_per_node[node]
        if cur >= max_groups_per_node:
            continue  # 已经够 5 组
        s, e = offsets[node], offsets[node+1]
        nbrs = neighbors[s:e]
        # 不限 reused，只要组数不足就补
        while cur < max_groups_per_node and nbrs.size >= group_size:
            sel = nbrs[:group_size]
            new_ng.append(sel.copy())
            new_src.append(node)
            slots += group_size
            # 更新 missing / used
            for v in sel:
                if used[v] == 0:
                    missing -= 1
                used[v] = 1
            nbrs = nbrs[group_size:]
            cur += 1
        # 如果已达到阈值也可以提前退出
        if slots + missing >= threshold:
            break

    if new_src:
        new_ng  = np.vstack(new_ng)
        new_src = np.array(new_src, dtype=np.int32)
    else:
        new_ng  = np.empty((0, group_size), dtype=np.int32)
        new_src = np.empty((0,),            dtype=np.int32)

    print(f"[第二阶段] 新增组: {new_src.size}")

    # 合并输出
    all_ng  = np.vstack([valid_ng, new_ng])
    all_src = np.concatenate([valid_src, new_src])
    final_slots = slots + missing
    print(f"最终存储槽位: {final_slots}, 膨胀率: {final_slots/total_nodes:.2f}×")

    return all_src, all_ng


# ================== 预计算分组生成 ==================
# 1. 加载并分析原始边关系
stats = ultra_fast_analysis(
    '/mnt/n3/papers100M-bin/processed/edge_index.npy'
)

# 2. 添加自环
stats2 = add_self_edges_all(stats, None)

# 3. 移除特定节点索引后的过滤
# /home/embed/Documents/gids/evaluation/pr_large_10.pt
# /home/embed/Documents/gids/evaluation/wq/pr_large_30.pt
remove_idx = torch.load(
    '/home/embed/Documents/gids/evaluation/freq_hotness.pt'
).numpy()
_, filtered = split_neighbors_by_remove_indices_enhanced(stats2, remove_idx)
# 4. 加载 PageRank 分数
prank = torch.load(
    '/home/embed/Documents/gids/evaluation/pr_papers100M.pt'
).numpy()

pre_srcs, pre_groups = full_optimized_grouping_multi_var_group_order(
    stats=filtered,
    prank_scores=prank,
    redundancy_rate=0.4,        # 总膨胀率 1 + 0.4 = 1.4×
    group_size=2,               # 每组 2 个邻居
    max_groups_per_node=5       # 每个节点最多 5 组
)


In [5]:
import numpy as np

# 加载 .npy 文件
labels = np.load('/mnt/n3/papers100M-bin/processed/node_label.npy')

# 将 (N, 1) 的数组变成 (N,) 一维
flat_labels = labels.reshape(-1)

# 检查 NaN 的占比
nan_mask = np.isnan(flat_labels)
num_nan = np.sum(nan_mask)
num_total = flat_labels.shape[0]

print(f"标签总数: {num_total:,}")
print(f"NaN 标签数量: {num_nan:,}")
print(f"NaN 占比: {num_nan / num_total:.2%}")

# 如果需要，可以打印几个非 NaN 的样本看看
valid_labels = flat_labels[~nan_mask]
print("\n前几个非 NaN 标签值:")
print(valid_labels[:10])



标签总数: 111,059,956
NaN 标签数量: 109,513,174
NaN 占比: 98.61%

前几个非 NaN 标签值:
[ 79.  26.  80.  35.  80. 118. 102. 132.  27.  88.]


In [6]:
import numpy as np

# 总节点数和有标签数量
total_nodes = 133_633_040
num_labeled = total_nodes // 100  # 1%

# 初始化所有行为 NaN
labels = np.full((total_nodes, 1), np.nan, dtype=np.float32)

# 随机选择 1% 节点作为有标签节点
labeled_indices = np.random.choice(total_nodes, num_labeled, replace=False)

# 生成随机标签（假设类别数为 150，可按需调整）
num_classes = 19
random_labels = np.random.randint(0, num_classes, size=(num_labeled, 1))

# 写入标签
labels[labeled_indices] = random_labels.astype(np.float32)

# 保存为 .npy 文件
np.save("/home/embed/Documents/Hyperion_notes/dataset/ukunion/node_label.npy", labels)

print(f"✅ 成功生成 node_label.npy")
print(f"总节点数: {total_nodes:,}")
print(f"有标签节点数: {num_labeled:,} ({num_labeled / total_nodes:.2%})")

✅ 成功生成 node_label.npy
总节点数: 133,633,040
有标签节点数: 1,336,330 (1.00%)
